### Imports

In [0]:
from google.colab import drive, files
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models,layers,losses,optimizers, backend
from sklearn.model_selection import train_test_split
import random
from sklearn.utils import class_weight
from scipy import ndimage as nd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Convolution2D, BatchNormalization, Activation,DepthwiseConv2D, Input, Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.metrics import binary_accuracy, mae
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import VGG16
import warnings
backend.set_image_data_format('channels_last')


In [0]:
drive.mount('/content/gdrive/')


## Downloading the dataset and saving it to drive

In [0]:
# !wget http://download.cs.stanford.edu/deep/MRNet-v1.0.zip
# !unzip -qq "MRNet-v1.0.zip" -d "/content/gdrive/My Drive/MRNetDataset/"

# Models

## VGG16

In [0]:
#VGG16
def vgg16():
  VGG16 = models.Sequential()

  VGG16.add(layers.Conv2D(input_shape=(256,256,3), filters=64, kernel_size=(3,3), padding='same', activation='relu'))
  VGG16.add(Dropout(0.2))
  VGG16.add(layers.Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
  VGG16.add(Dropout(0.2))

  VGG16.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))

  VGG16.add(layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
  VGG16.add(Dropout(0.2))
  VGG16.add(layers.Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
  VGG16.add(Dropout(0.2))

  VGG16.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))

  VGG16.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
  VGG16.add(Dropout(0.2))
  VGG16.add(layers.Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu'))
  VGG16.add(Dropout(0.2))

  VGG16.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))

  VGG16.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
  VGG16.add(Dropout(0.2))
  VGG16.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
  VGG16.add(Dropout(0.2))
  VGG16.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
  VGG16.add(Dropout(0.2))

  VGG16.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))

  VGG16.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
  VGG16.add(Dropout(0.2))
  VGG16.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
  VGG16.add(Dropout(0.2))
  VGG16.add(layers.Conv2D(filters=512, kernel_size=(3,3), padding='same', activation='relu'))
  VGG16.add(Dropout(0.2))

  VGG16.add(layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))

  VGG16.add(layers.Flatten())

  VGG16.add(layers.Dense(units=4096, activation='relu'))
  VGG16.add(Dropout(0.5))
  VGG16.add(layers.Dense(units=4096, activation='relu'))
  VGG16.add(Dropout(0.5))
  VGG16.add(layers.Dense(units=1000, activation='relu'))
  VGG16.add(Dropout(0.5))
  VGG16.add(layers.Dense(units=1, activation='sigmoid'))

  SGD = optimizers.SGD(learning_rate=0.1) #default learning rate is 0.001
  VGG16.compile(optimizer= SGD, loss = losses.binary_crossentropy, metrics=['accuracy']) #Should be F1_Score / Recall

  return VGG16

## ResNet

In [0]:
#ResNet
#Can't use Sequential model because we need to skip connections

def add_shortcut(X, f, downsample=False):
  shortcut = X
  if downsample:
    s = (2,2)
    shortcut = layers.Conv2D(filters=f, kernel_size=(3,3), strides=s, padding='same')(shortcut) 
    #need to revise padding to make sure input tensors are sized correctly #TODO
    shortcut = layers.BatchNormalization(axis = 3)(shortcut)
  else:
    s = (1,1)

  X = layers.Conv2D(filters=f, kernel_size=(3,3), strides=s, padding='same')(X)
  X = layers.BatchNormalization(axis = 3)(X)
  X = layers.Activation('relu')(X)
  X = Dropout(0.2)(X)

  X = layers.Conv2D(filters=f, kernel_size=(3,3), padding='same')(X)
  X = layers.BatchNormalization(axis = 3)(X)
  X = layers.Activation('relu')(X)
  X = Dropout(0.2)(X)

  X = layers.Add()([X, shortcut])
  X = layers.Activation('relu')(X)

  return X



def resnet():
  inputLayer = layers.Input((256,256,3))
  X = layers.ZeroPadding2D((3,3))(inputLayer)
  X = layers.Conv2D(filters=64, kernel_size=(7,7), strides=(2,2))(X)
  X = layers.BatchNormalization(axis = 3)(X)
  X = layers.Activation('relu')(X)
  X = Dropout(0.2)(X)
  X = layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

  for _ in range(3):
    X = add_shortcut(X, 64)

  X = add_shortcut(X, 128, downsample = True)
  for _ in range(2):
    X = add_shortcut(X, 128)

  X = add_shortcut(X, 256, downsample = True)
  for _ in range(2):
    X = add_shortcut(X, 256)

  X = add_shortcut(X, 512, downsample = True)
  for _ in range(2):
    X = add_shortcut(X, 512)

  X = layers.AveragePooling2D((3, 3), strides=(2, 2))(X)
  X = layers.Flatten()(X)
  X = layers.Dense(units=1, activation='sigmoid')(X)

  ResNet = models.Model(inputs=inputLayer, outputs=X)

  SGD = optimizers.SGD(learning_rate = 0.1) #default learning rate is 0.001
  ResNet.compile(optimizer= SGD, loss = losses.binary_crossentropy, metrics=['accuracy'])
  return ResNet



## InceptionV3

In [0]:
#InceptionV3

def createBranch(X, f, k, p='same', s=(1,1)):
  X = layers.Conv2D(filters=f, kernel_size=k, strides=s, padding=p, kernel_regularizer=l2(0.01))(X)
  X = layers.BatchNormalization(axis = 3)(X)
  X = layers.Activation('relu')(X)
  X = Dropout(0.2)(X)
  return X

def inception():

  inputLayer = layers.Input((256,256,3))
  X = createBranch(inputLayer, 32, (3,3), p='valid')
  X = createBranch(X, 64, (3,3))
  X = layers.MaxPooling2D((3,3), strides=(2,2))(X)

  X = createBranch(X, 80, (1, 1), p='valid')
  X = createBranch(X, 192, (3, 3), p='valid')
  X = layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

  B1 = createBranch(X, 64, (1, 1))

  B5 = createBranch(X, 48, (1, 1))
  B5 = createBranch(B5, 64, (5, 5))

  B3 = createBranch(X, 64, (1, 1))
  B3 = createBranch(B3, 96, (3, 3))
  B3 = createBranch(B3, 96, (3, 3))

  Pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(X)
  Pool = createBranch(Pool, 32, (1, 1))
  X = layers.concatenate([B1, B5, B3, Pool],axis=3)



  B1 = createBranch(X, 64, (1, 1))

  B5 = createBranch(X, 48, (1, 1))
  B5 = createBranch(B5, 64, (5, 5))

  B3 = createBranch(X, 64, (1, 1))
  B3 = createBranch(B3, 96, (3, 3))
  B3 = createBranch(B3, 96, (3, 3))

  Pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(X)
  Pool = createBranch(Pool, 64, (1, 1))
  X = layers.concatenate([B1, B5, B3, Pool],axis=3)


  B1 = createBranch(X, 64, (1, 1))

  B5 = createBranch(X, 48, (1, 1))
  B5 = createBranch(B5, 64, (5, 5))

  B3 = createBranch(X, 64, (1, 1))
  B3 = createBranch(B3, 96, (3, 3))
  B3 = createBranch(B3, 96, (3, 3))

  Pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(X)
  Pool = createBranch(Pool, 64, (1, 1))
  X = layers.concatenate([B1, B5, B3, Pool],axis=3)


  B3 = createBranch(X, 384, (3, 3), s=(2, 2), p='valid')

  B3branch = createBranch(X, 64, (1, 1))
  B3branch = createBranch(B3branch, 96, (3, 3))
  B3branch = createBranch(B3branch, 96, (3, 3), s=(2, 2), p='valid')

  Pool = layers.MaxPooling2D((3, 3), strides=(2, 2))(X)
  X = layers.concatenate([B3, B3branch, Pool], axis=3)



  B1 = createBranch(X, 192, (1, 1))

  B7 = createBranch(X, 128, (1, 1))
  B7 = createBranch(B7, 128, (1, 7))
  B7 = createBranch(B7, 192, (7, 1))

  B7branch = createBranch(X, 128, (1, 1))
  B7branch = createBranch(B7branch, 128, (7, 1))
  B7branch = createBranch(B7branch, 128, (1, 7))
  B7branch = createBranch(B7branch, 128, (7, 1))
  B7branch = createBranch(B7branch, 192, (1, 7))

  Pool = layers.AveragePooling2D((3, 3), strides=(1, 1), padding='same')(X)
  Pool = createBranch(Pool, 192, (1, 1))

  X = layers.concatenate([B1, B7, B7branch, Pool],axis=3)


  X = layers.GlobalAveragePooling2D()(X)
  X = layers.Dense(units=1, activation='sigmoid')(X)

  Inception = models.Model(inputs=inputLayer, outputs=X)
  SGD = optimizers.SGD(learning_rate=0.1) #default learning rate is 0.001
  Inception.compile(optimizer= SGD, loss = losses.binary_crossentropy, metrics=['accuracy']) 

  return Inception




## MobileNet

In [0]:
#MobileNet

def mobilenet():

  inputLayer = Input((256,256, 3))

  X = Convolution2D(32, (3, 3), strides=(2, 2), padding='same', use_bias=False, kernel_regularizer=l2(0.01))(inputLayer)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Dropout(0.2)(X)

  X = DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Convolution2D(64, (1, 1), strides=(1, 1), padding='same', use_bias=False, kernel_regularizer=l2(0.01))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Dropout(0.2)(X)

  X = DepthwiseConv2D(kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=False)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Convolution2D(128, (1, 1), strides=(1, 1), padding='same', use_bias=False, kernel_regularizer=l2(0.01))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Dropout(0.2)(X)

  X = DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Convolution2D(128, (1, 1), strides=(1, 1), padding='same', use_bias=False, kernel_regularizer=l2(0.01))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Dropout(0.2)(X)

  X = DepthwiseConv2D(kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=False)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Convolution2D(256, (1, 1), strides=(1, 1), padding='same', use_bias=False, kernel_regularizer=l2(0.01))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Dropout(0.2)(X)

  X = DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Convolution2D(256, (1, 1), strides=(1, 1), padding='same', use_bias=False, kernel_regularizer=l2(0.01))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Dropout(0.2)(X)

  X = DepthwiseConv2D(kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=False)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Convolution2D(512, (1, 1), strides=(1, 1), padding='same', use_bias=False, kernel_regularizer=l2(0.01))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Dropout(0.2)(X)

  X = DepthwiseConv2D(kernel_size=(3, 3), strides=(2, 2), padding='same', use_bias=False)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Convolution2D(1024, (1, 1), strides=(1, 1), padding='same', use_bias=False, kernel_regularizer=l2(0.01))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Dropout(0.2)(X)

  X = DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), padding='same', use_bias=False)(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Convolution2D(1024, (1, 1), strides=(1, 1), padding='same', use_bias=False, kernel_regularizer=l2(0.01))(X)
  X = BatchNormalization()(X)
  X = Activation('relu')(X)
  X = Dropout(0.2)(X)

  X = GlobalAveragePooling2D()(X)
  X = Dense(units=1, activation='sigmoid', kernel_regularizer=l2(0.01))(X)

  MobileNet = models.Model(inputs=inputLayer, outputs=X)
  SGD = optimizers.SGD(learning_rate=0.1) #default learning rate is 0.001
  MobileNet.compile(optimizer= SGD, loss = losses.binary_crossentropy, metrics=['accuracy'])

  return MobileNet


#Training

In [0]:
warnings.simplefilter('ignore')
def model_train(basepath, plane, modelname, label):
  path = basepath + "/" + plane
  files = [f for f in sorted(os.listdir(path)) if f.endswith(".npy")]
  count = 10
  labels = np.load(basepath + "/" + label + ".npy")[:,1]
  print(labels.shape)
  mc = ModelCheckpoint("/content/gdrive/My Drive/Models/"+modelname+"_"+plane+"_"+label+".h5", monitor='val_accuracy',mode='max', save_best_only=True)
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=5)
  lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=6, min_lr=0.0001)
  batch = 32
  stride = 113
  validation = None
  for i in range(1,count):
    print(i)
    pre_process = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
    pre_process_valid = ImageDataGenerator(featurewise_center=True, featurewise_std_normalization=True)
    X_train = None
    Y_train = None
    cases= files[(i*stride):(i*stride)+stride]
    for j,filename in enumerate(cases):
      totalpath = path + "/" + filename
      temp = np.load(totalpath)
      if X_train is None:
        X_train = temp
        Y_train = np.repeat(labels[(i*stride)+j], temp.shape[0])
      else:
        X_train = np.vstack((X_train, temp))
        Y_train = np.append(Y_train, np.repeat(labels[(i*stride)+j], temp.shape[0]))

    print(Y_train.shape)
    X_train = np.array(X_train).reshape((len(X_train),256,256))
    print(X_train.shape)

    X_train =np.stack([X_train]*3,axis=3)

    X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=0.1, shuffle=True)


    if validation is None:
      validation = X_valid
      validation_labels = Y_valid
    else:
      validation = np.vstack((validation, X_valid))
      validation_labels = np.append(validation_labels, Y_valid)

      
    pre_process_valid.fit(validation)
    pre_process.fit(X_train)

    print(X_train.shape)
    print(Y_train.shape)

    try:
      model = models.load_model("/content/gdrive/My Drive/Models/"+modelname+"_"+plane+"_"+label+".h5")
      print("Loading Model")
    except OSError:
      print("Generating new Model")
      model = model_gen(modelname)

    weights = class_weight.compute_class_weight('balanced',np.unique(Y_train), Y_train)
    class_weights = dict(enumerate(weights))
    validation_weights = class_weight.compute_sample_weight('balanced', validation_labels)
  

    history = model.fit(pre_process.flow(X_train,Y_train, batch_size=batch),
                        validation_data=pre_process_valid.flow(validation, validation_labels),
                        class_weight = class_weights,epochs = 50, validation_steps = len(validation)/batch,
                        callbacks = [mc, es, lr], steps_per_epoch = len(X_train)/batch)

def model_gen(modelname):
  return eval(modelname+"()")

model_train("/content/gdrive/My Drive/Dataset/TrainingSet","axial","resnet", "meniscus")

## Our Approaches

In [0]:

def define_labels():   #Generating Mutually Exclusive Labels to account for intersections between the 3 original ones (Didn't produce promising results)
  base_path = "/content/gdrive/My Drive/Dataset/"
  path = base_path + "TrainingSet/"
  # valid_path = base_path + "ValidationSet/"
  # test_path = base_path + "TestSet/"
  abnormal = np.load(path+"abnormal.npy")
  acl = np.load(path+"acl.npy")
  meniscus = np.load(path+"meniscus.npy")
  cases = abnormal.shape[0]
  classes = np.zeros((cases, 5))
  for i in range(cases):
      if(abnormal[i,1] == 1 and acl[i,1] == 1 and meniscus[i,1] == 1):
        classes[i,4] = 1
      elif(abnormal[i,1] == 1 and meniscus[i,1] == 1):
        classes[i,3] = 1
      elif(abnormal[i,1] == 1 and acl[i,1] == 1):
        classes[i,2] = 1
      elif(abnormal[i,1] == 1):
        classes[i,1] = 1
      elif(abnormal[i,1] == 0 and acl[i,1] == 0 and meniscus[i,1] == 0):
        classes[i,0] = 1
  np.save(path+"total.npy",classes)
  

def load_dataset(basepath, plane):   #Using each case as a datapoint, padding each case to ensure constant tensor shape
  MAX = 61 #max number of slices
  path = basepath + "/" + plane
  files = [f for f in sorted(os.listdir(path)) if f.endswith(".npy")]
  dataset = []
  for filename in files:
    totalpath = path + "/" + filename
    temp = np.load(totalpath)
    temp = 1/255*temp
    pad = MAX - temp.shape[0]
    for _ in range(pad):
      temp = np.vstack((temp, temp[-1,:,:].reshape((1,256,256))))
    dataset.append(temp)
  
  size = len(dataset)
  dataset = np.array(dataset)
  dataset = np.swapaxes(dataset, 1,2)
  dataset = np.swapaxes(dataset, 2,3)
  # np.save(basepath+"/"+plane.upper()+".npy", dataset)

  
  
  print(dataset.shape)
  return dataset

def load_interdataset(basepath,plane):  #interpolating across slices of each case, was unrealistic in the given time limit
  MIN = 19
  path = basepath + "/" + plane
  files = [f for f in sorted(os.listdir(path)) if f.endswith(".npy")]
  dataset = []
  for filename in files:
    totalpath = path + "/" + filename
    temp = np.load(totalpath)
    factors = [w/float(f) for w,f in zip([MIN,256,256], temp.shape)]
    interpolated_scan = nd.interpolation.zoom(temp, zoom=factors)
    dataset.append(interpolated_scan)
  
  
  dataset = np.array(dataset)
  dataset = np.swapaxes(dataset, 1,2)
  dataset = np.swapaxes(dataset, 2,3)

  print(dataset.shape)
  return dataset


def load_reducedDataset(basepath, plane):  #using 3 slices from each case (Not enough data, too many epochs needed)
  path = basepath + "/" + plane
  files = [f for f in sorted(os.listdir(path)) if f.endswith(".npy")]
  dataset = []
  for filename in files:
    totalpath = path + "/" + filename
    temp = np.load(totalpath)
    mid_slice = temp.shape[0]//2
    case = np.array([temp[mid_slice-5], temp[mid_slice], temp[mid_slice+5]])
    dataset.append(case)

  dataset = np.array(dataset)
  dataset = np.swapaxes(dataset, 1,2)
  dataset = np.swapaxes(dataset, 2,3)

  print(dataset.shape)
  return dataset

def load_large(base_path, labels, plane):  #considering each slice as a datapoint as specified in the requirements
    path = base_path + "/" + plane
    files = [f for f in sorted(os.listdir(path)) if f.endswith(".npy")]
    labels = np.load(base_path + "/" + labels + ".npy")[:,1]
    Y = None
    X = None
    for i,filename in enumerate(files):
      totalpath = path + "/" + filename
      temp = np.load(totalpath)
      if X is None:
        X = temp
        Y = np.repeat(labels[i], temp.shape[0])
      else:
        X = np.vstack((X, temp))
        Y = np.append(Y, np.repeat(labels[i], temp.shape[0]))
    return X, Y
  


      





## Ensemble

In [0]:
from sklearn.metrics import accuracy_score

def load_test(basepath, plane, label):
  path = basepath + "/" + plane
  files = [f for f in sorted(os.listdir(path)) if f.endswith(".npy")]
  slices = []
  X = None
  Y= None
  labels = np.load(basepath+"/"+label+".npy")[:,1]
  for i,filename in enumerate(files):
    totalpath = path + "/" + filename
    temp = np.load(totalpath)
    slices.append(temp.shape[0])
    if X is None:
      X = temp
      Y = np.repeat(labels[i], temp.shape[0])
    else:
      X = np.vstack((X, temp))
      Y = np.append(Y, np.repeat(labels[i], temp.shape[0]))
  
  X =np.stack([X]*3,axis=3) #This is because axial models were not trained with RGB stacking
  return X, Y, slices
    

def predict_case(preds, slices):
  i = 0
  finals = []
  for case in slices:
    final = round(np.sum(preds[i:i+case])/case)
    finals.append(final)
    i += case
  # print(len(finals))
  return np.array(finals).astype(np.int_)




def ensemble(model, label):
  axial_model = models.load_model("/content/gdrive/My Drive/Models/"+model+"_axial_"+label+".h5")
  coronal_model = models.load_model("/content/gdrive/My Drive/Models/"+model+"_coronal_"+label+".h5")
  sagittal_model = models.load_model("/content/gdrive/My Drive/Models/"+model+"_sagittal_"+label+".h5")

  X_coronal, Y_coronal, slices_coronal = load_test("/content/gdrive/My Drive/Dataset/TestSet", "coronal", label)
  X_sagittal, Y_sagittal, slices_sagittal = load_test("/content/gdrive/My Drive/Dataset/TestSet", "sagittal", label)
  X_axial, Y_axial, slices_axial = load_test("/content/gdrive/My Drive/Dataset/TestSet", "axial", label)

  w1 = axial_model.evaluate(X_axial, Y_axial)[1]
  axial_predictions = (axial_model.predict(X_axial) > 0.5).astype(np.int_).ravel()

  axial_predictions = predict_case(axial_predictions, slices_axial)

  w2 = coronal_model.evaluate(X_coronal, Y_coronal)[1]
  coronal_predictions = (coronal_model.predict(X_coronal) > 0.5).astype(np.int_).ravel()

  coronal_predictions = predict_case(coronal_predictions, slices_coronal)

  w3 = sagittal_model.evaluate(X_sagittal, Y_sagittal)[1]
  sagittal_predictions = (sagittal_model.predict(X_sagittal) > 0.5).astype(np.int_).ravel()

  sagittal_predictions = predict_case(sagittal_predictions, slices_sagittal)



  print(axial_predictions.shape)
  print(axial_predictions)
  print(Y_axial)
  print(coronal_predictions.shape)
  print(sagittal_predictions.shape)


def ensemble_evaluate(modelname,label):
  planes = ["axial", "coronal", "sagittal"]
  plane_predictions = None
  weights = []
  for i,plane in enumerate(planes):
    print("Predicting plane: " + plane)
    # print("Loading Model")
    model = models.load_model("/content/gdrive/My Drive/Models/"+modelname+"_"+plane+"_"+label+".h5")
    # print("Loading Data")
    X_test, Y_test, slices = load_test("/content/gdrive/My Drive/Dataset/TestSet", plane, label)
    # print("Evaluating")
    weights.append(model.evaluate(X_test, Y_test, verbose=0)[1])
    predictions = (model.predict(X_test)>0.5).astype(np.int_).ravel()
    # print("Predicting")
    if plane_predictions is None:
      plane_predictions = predict_case(predictions, slices)
    else:
      plane_predictions = np.vstack((plane_predictions, predict_case(predictions, slices)))
    # print(plane_predictions.shape)

  weighted_predictions = np.arange(plane_predictions.shape[1])
  weights = np.array(weights)
  for i in range(plane_predictions.shape[1]):
    weighted_predictions[i] = np.argmax(np.bincount(np.array([plane_predictions[0][i], plane_predictions[1][i], plane_predictions[2][i]]),weights))
  

  ground_truth = np.load("/content/gdrive/My Drive/Dataset/TestSet/"+label+".npy")[:,1]

  print("Accuracy of " + modelname + " model with " + label + " label: ")
  print(accuracy_score(ground_truth, weighted_predictions))
  






In [0]:
ensemble_evaluate("resnet","abnormal")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of resnet model with abnormal label: 
0.7666666666666667


In [0]:
ensemble_evaluate("resnet","acl")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of resnet model with acl label: 
0.55


In [0]:
ensemble_evaluate("resnet","meniscus")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of resnet model with meniscus label: 
0.5666666666666667


In [0]:
ensemble_evaluate("inception","abnormal")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of inception model with abnormal label: 
0.7916666666666666


In [0]:
ensemble_evaluate("inception","acl")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of inception model with acl label: 
0.45


In [0]:
ensemble_evaluate("inception","meniscus")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of inception model with meniscus label: 
0.43333333333333335


In [0]:
ensemble_evaluate("vgg16","abnormal")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of vgg16 model with abnormal label: 
0.7916666666666666


In [0]:
ensemble_evaluate("vgg16","acl")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of vgg16 model with acl label: 
0.55


In [0]:
ensemble_evaluate("vgg16","meniscus")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of vgg16 model with meniscus label: 
0.5666666666666667


In [0]:
ensemble_evaluate("mobilenet","abnormal")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of mobilenet model with abnormal label: 
0.7916666666666666


In [0]:
ensemble_evaluate("mobilenet","acl")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of mobilenet model with acl label: 
0.55


In [0]:
ensemble_evaluate("mobilenet","meniscus")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of mobilenet model with meniscus label: 
0.43333333333333335


In [0]:
ensemble_evaluate("TransferVGG","abnormal")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of TransferVGG model with abnormal label: 
0.7916666666666666


In [0]:
ensemble_evaluate("TransferVGG","acl")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of TransferVGG model with acl label: 
0.55


In [0]:
ensemble_evaluate("TransferVGG","meniscus")

Predicting plane: axial
Predicting plane: coronal
Predicting plane: sagittal
Accuracy of TransferVGG model with meniscus label: 
0.5833333333333334
